# Submission using XGBoost

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_pickle('input/train.p')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_vect,q2_vect
0,0,1,2,step step guide invest share market india,step step guide invest share market,0,"(0, 1166933)\t0.273417212577\n (0, 1166849)...","(0, 1166933)\t0.290476597177\n (0, 1166849)..."
1,1,3,4,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0,"(0, 1173397)\t0.384783614373\n (0, 1173165)...","(0, 1365788)\t0.156962278725\n (0, 1364462)..."
2,2,5,6,increase speed internet connection using vpn,internet speed increased hacking dns,0,"(0, 1319028)\t0.289711244937\n (0, 1299405)...","(0, 1149988)\t0.420070820689\n (0, 1149702)..."
3,3,7,8,mentally lonely solve,find remainder math 23 24 math divided 2423,0,"(0, 1136891)\t0.298113614203\n (0, 784899)\...","(0, 1029011)\t0.278515360209\n (0, 1028954)..."
4,4,9,10,one dissolve water quickly sugar salt methane ...,fish would survive salt water,0,"(0, 1328636)\t0.273561526358\n (0, 1327947)...","(0, 1367335)\t0.391369076414\n (0, 1364462)..."


## Prepare data

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from utils import parallelize_dataframe

In [6]:
def question_cosine_similarity(df):
    return cosine_similarity(df['q1_vect'], df['q2_vect'])[0]

In [7]:
# %time df = parallelize_dataframe(df, question_cosine_similarity)
%time df['tf_similarity'] = df.apply(question_cosine_similarity, axis=1)['id']

CPU times: user 25min 14s, sys: 5min 40s, total: 30min 55s
Wall time: 31min 32s


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
        df[training_fields], df['is_duplicate'], test_size=0.2, random_state=42)

## Training XGBoost model

In [17]:
import xgboost as xgb

In [18]:
# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 5

In [19]:
training_fields = ['tf_similarity']

In [20]:
d_train = xgb.DMatrix(X_train[training_fields], label=y_train)
d_valid = xgb.DMatrix(X_test[training_fields], label=y_test)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 200, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.68855	valid-logloss:0.688605
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.651038	valid-logloss:0.651576
[20]	train-logloss:0.624778	valid-logloss:0.625715
[30]	train-logloss:0.605804	valid-logloss:0.607082
[40]	train-logloss:0.591846	valid-logloss:0.593413
[50]	train-logloss:0.58142	valid-logloss:0.583256
[60]	train-logloss:0.573565	valid-logloss:0.575628
[70]	train-logloss:0.567579	valid-logloss:0.569827
[80]	train-logloss:0.562998	valid-logloss:0.565424
[90]	train-logloss:0.559483	valid-logloss:0.562069
[100]	train-logloss:0.556777	valid-logloss:0.559498
[110]	train-logloss:0.554681	valid-logloss:0.55753
[120]	train-logloss:0.553062	valid-logloss:0.556028
[130]	train-logloss:0.551799	valid-logloss:0.554883
[140]	train-logloss:0.550812	valid-logloss:0.554008
[150]	train-logloss:0.550049	valid-logloss:0.553342
[160]	train-logloss:0.549452	va

## Submission

In [23]:
df_test = pd.read_csv('input/test.csv')
df_test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [22]:
d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test)